In [45]:
import cosmology
import fv 
import my_fv 

import fd
import fd_1d
import fd_2d
import numpy as np
import tree
import interpolation 
import config as configuration
import integration 
import schemes 


from enum import Enum
import matplotlib.pyplot as plt

class FluidScheme(schemes.SchroedingerScheme):
    def __init__(self, config, generateIC):
        super().__init__(config, generateIC)

        #Fluid scheme-specific settings
        self.fluidMode        = config["fluidMode"]
        self.useSlopeLimiting = True
        self.vol              = self.dx**2

        if self.dimension != 2:
            raise ValueError("Fluid scheme only implemented in 2D")

        self.density = np.abs(self.psi) ** 2
        self.potential =  self.computePotential(self.density)
        self.phase   = fd.make_continuous(np.angle(self.psi))

        self.vx, self.vy                = fd_2d.getC2Gradient(self.phase, self.dx)
        self.P                          = fd.getQuantumPressure(self.density, self.dx, 1)
        self.Mass, self.Momx, self.Momy = fv.getConserved(self.density, self.vx, self.vy, self.vol)

        self.vmax = np.maximum(np.max(np.abs(self.vx)), np.max(np.abs(self.vy)))
        self.cfl  = .4


    def step(self, dt):
        if self.outputTimestep:
            print(self.t, dt)
        dx = self.dx
        Mass, Momx, Momy, vol, maxSpeed = self.Mass, self.Momx, self.Momy, self.vol, self.vmax

        density, vx, vy = fv.getPrimitive(Mass, Momx, Momy, vol)

        ###KICK###
        # Add Source (half-step)

        #Here V is the updated V at t = t, maybe this introduces an error of order dt?
        Vx, Vy = fd_2d.getCenteredGradient(self.potential, dx)

        print("Vx, Vy: ", Vx, Vy)

        Mass, Momx, Momy = fv.addSourceTerm(Mass, Momx, Momy, Vx, Vy, dt/2)

        ###DRIFT###
        density, vx, vy    = fv.getPrimitive( Mass, Momx, Momy, vol )

        self.vmax = 0.1#np.maximum(np.max(np.abs(vx)), np.max(np.abs(vy))) +  .5/dx
        print("drift density, drift vx, drift vy ", density, vx, vy)
        
        P              = fd_2d.getC2QuantumPressure(density, dx)
        P_dx,   P_dy   = fd_2d.getCenteredGradient(P,   dx)
        density_dx, density_dy = fd_2d.getCenteredGradient(density, dx)
        vx_dx,  vx_dy  = fd_2d.getCenteredGradient(vx,  dx)
        vy_dx,  vy_dy  = fd_2d.getCenteredGradient(vy,  dx)


        # slope limit gradients
        #if self.useSlopeLimiting:
        #    density_dx, density_dy = fv.slopeLimit(density, dx, density_dx, density_dy)
        #    vx_dx,  vx_dy  = fv.slopeLimit(vx , dx, vx_dx,  vx_dy )
        #    vy_dx,  vy_dy  = fv.slopeLimit(vy , dx, vy_dx,  vy_dy )
        #    P_dx,   P_dy   = fv.slopeLimit(P  , dx, P_dx,   P_dy  )

        print("pressure gradients: ", P_dx, P_dy)

        # extrapolate half-step in time
        density_prime = density - 0.5*dt * ( vx * density_dx + density * vx_dx + vy * density_dy + density * vy_dy)
        vx_prime  = vx  - 0.5*dt * ( vx * vx_dx + vy * vx_dy + P_dx )
        vy_prime  = vy  - 0.5*dt * ( vx * vy_dx + vy * vy_dy + P_dy )

        print("density prime, vx_prime, vy_prime", density_prime, vx_prime, vy_prime)

        # extrapolate in space to face centers
        density_XL, density_XR, density_YL, density_YR = fv.extrapolateInSpaceToFace(density_prime, density_dx, density_dy, dx)
        vx_XL,  vx_XR,  vx_YL,  vx_YR  = fv.extrapolateInSpaceToFace(vx_prime,  vx_dx,  vx_dy,  dx)
        vy_XL,  vy_XR,  vy_YL,  vy_YR  = fv.extrapolateInSpaceToFace(vy_prime,  vy_dx,  vy_dy,  dx)


        print("extrapolated density, vxext, vyext ", density_XL, density_XR, density_YL, density_YR, vx_XL,  vx_XR,  vx_YL,  vx_YR, vy_XL,  vy_XR,  vy_YL,  vy_YR)


        # compute fluxes (local Lax-Friedrichs/Rusanov)
        flux_Mass_X, flux_Momx_X, flux_Momy_X = fv.getFlux(density_XL, density_XR, vx_XL, vx_XR, vy_XL, vy_XR, dx, 0, self.vmax)
        flux_Mass_Y, flux_Momy_Y, flux_Momx_Y = fv.getFlux(density_YL, density_YR, vy_YL, vy_YR, vx_YL, vx_YR, dx, 1, self.vmax)

        print("flux mass x, flux mass y", flux_Mass_X, flux_Mass_Y)

        print("flux momx x, flux momx y, flux momy x, flux momy y ", flux_Momx_X, flux_Momx_Y, flux_Momy_X, flux_Momy_Y)
        
        # update solution
        Mass   = fv.applyFluxes(Mass, flux_Mass_X, flux_Mass_Y, dx, dt)
        Momx   = fv.applyFluxes(Momx, flux_Momx_X, flux_Momx_Y, dx, dt)
        Momy   = fv.applyFluxes(Momy, flux_Momy_X, flux_Momy_Y, dx, dt)

        print("Mass, Momx, Momy", Mass, Momx, Momy)

        ####KICK###

        ##Get primitive variables
        density, vx, vy = fv.getPrimitive(Mass, Momx, Momy, vol)

        ##Calculate gravitational potential
        self.potential =  self.computePotential(density)
        Vx, Vy = fd_2d.getCenteredGradient(self.potential, dx)

        ## Add Source (half-step)
        Mass, Momx, Momy = fv.addSourceTerm(Mass, Momx, Momy, Vx, Vy, dt/2 )

        ##Get primitive variables
        self.density, self.vx, self.vy = fv.getPrimitive(Mass, Momx, Momy, vol)
        
        self.t += dt 

In [51]:
import my_fv 
import fd 

def getCenteredGradient(f, dx):
    gradient = np.zeros((f.ndim, *f.shape))
    for i in range(f.ndim):
        gradient[i] = fd.getCenteredGradient(f, dx, axis = i)
    return gradient


def extrapolateInSpaceToFace(f, f_grad, dx):
    extrapolatedf = np.zeros((f.ndim, 2, *f.shape))

    print(f.ndim)

    for i in range(f.ndim):
        f_iL = f - f_grad[i] * dx/2
        f_iL = np.roll(f_iL, fd.ROLL_R, axis=i)
        f_iR = f + f_grad[i] * dx/2

        extrapolatedf[i, 0] = f_iL
        extrapolatedf[i, 1] = f_iR

    return extrapolatedf

def getFlux(density_ext, velocities_ext, dx, axis, maxSpeed):
    # compute star (averaged) states
    rho_star     = 0.5 * (density_ext[axis, 0] + density_ext[axis, 1])
    momenta_star = np.zeros((rho_star.ndim, *rho_star.shape))

    for i in range(rho_star.ndim):
        momenta_star[i] = 0.5 * (density_ext[axis, 0] * velocities_ext[i, axis, 0] + density_ext[axis, 1] * velocities_ext[i, axis, 1])

    print("rho star, px star, py star ", rho_star, momenta_star[0], momenta_star[1])

    if axis ==  0:
        P1 = fv.getC2PressureTensor(rho_star, dx, 0, 0)
        P2 = fv.getC2PressureTensor(rho_star, dx, 1, 0)
    else:
        P1 = fv.getC2PressureTensor(rho_star, dx, 1, 1)
        P2 = fv.getC2PressureTensor(rho_star, dx, 0, 1)

    # compute fluxes (local Lax-Friedrichs/Rusanov)
    flux_Mass   = momenta_star[0]

    flux_Momenta = np.zeros((rho_star.ndim, *rho_star.shape))

    flux_Momenta[0]   = momenta_star[0]**2 / rho_star + P1
    flux_Momenta[1]   = momenta_star[0] * momenta_star[1] / rho_star + P2

    print("flux mass, flux momx, flux momy ", flux_Mass, flux_Momenta[0], flux_Momenta[1])
    print("rho_l rho_r ", density_ext[axis, 0], density_ext[axis, 1])
    print("vx_l vx_r ", velocities_ext[0, axis, 0], velocities_ext[0, axis, 1])
    print("vy_l vy_r ", velocities_ext[1, axis, 0], velocities_ext[1, axis, 1])

    maxSpeed = 4
    
    # add stabilizing diffusive term
    flux_Mass   = flux_Mass - maxSpeed * 0.5 * (density_ext[axis, 0] - density_ext[axis, 1])

    for i in range(rho_star.ndim):
        flux_Momenta[i]   -= maxSpeed * 0.5 * (density_ext[axis, 0] * velocities_ext[i, axis, 0] - density_ext[axis, 1] * velocities_ext[i, axis, 1])

    print("stabilised flux mass, flux momx, flux momy ", flux_Mass, flux_Momenta[0], flux_Momenta[1])

    return flux_Mass, flux_Momenta



def mapplyFluxes(F, flux_F, dx, dt):
    """
    Apply fluxes to conserved variables
    F        is a matrix of the conserved variable field
    flux_F_X is a matrix of the x-dir fluxes
    flux_F_Y is a matrix of the y-dir fluxes
    dx       is the cell size
    dt       is the timestep
    """

    # update solution
    for i in range(F.ndim):
        F += - dt * dx * flux_F[i]
        F +=   dt * dx * np.roll(flux_F[i], fd.ROLL_L,axis=i)

    return F



def applyFluxes(F, flux_F_X, flux_F_Y, dx, dt):
    """
    Apply fluxes to conserved variables
    F        is a matrix of the conserved variable field
    flux_F_X is a matrix of the x-dir fluxes
    flux_F_Y is a matrix of the y-dir fluxes
    dx       is the cell size
    dt       is the timestep
    """

    # update solution
    F += - dt * dx * flux_F_X
    F +=   dt * dx * np.roll(flux_F_X, fd.ROLL_L,axis=0)
    F += - dt * dx * flux_F_Y
    F +=   dt * dx * np.roll(flux_F_Y, fd.ROLL_L,axis=1)

    return F

class MUSCLHancock(schemes.SchroedingerScheme):
    def __init__(self, config, generateIC):
        super().__init__(config, generateIC)

        #Fluid scheme-specific settings
        self.fluidMode        = config["fluidMode"]

        self.density            = np.abs(self.psi) ** 2
        self.potential          = self.computePotential(self.density)
        self.potentialGradients = getCenteredGradient(self.potential, self.dx)
        self.phase              = fd.make_continuous(np.angle(self.psi))
        self.velocities         = getCenteredGradient(self.phase, self.dx)

    def step(self, dt):
        dx = self.dx
        volume = self.dx**2

        velocities, density, potentialGradients = self.velocities, self.density, self.potentialGradients

        #for extrapolation to face
        ddensity          = np.zeros(density.shape)
        dvelocities       = np.zeros((self.dimension, *density.shape))

        #after extrapolation
        velocities_prime  = np.zeros((self.dimension,                    *density.shape))
        velocityGradients = np.zeros((self.dimension, self.dimension,    *density.shape))
        massFluxes        = np.zeros((self.dimension,                    *density.shape))
        momentaFluxes     = np.zeros((self.dimension, self.dimension,    *density.shape))
        velocities_ext    = np.zeros((self.dimension, self.dimension, 2, *density.shape))

        ###KICK###

        print("Vx, Vy: ", potentialGradients[0], potentialGradients[1])

        # Add Source (half-step)
        velocities -= dt/2 * potentialGradients

        ###DRIFT###
        #extrapolate to face centers (half step in space using centered gradients (MUSCL) and half step in time (Hancock))

        self.vmax = np.max(np.abs(velocities)) +  .5/dx
        
        print("drift density, drift vx, drift vy ", density, velocities[0], velocities[1])

        P                 = fd_2d.getC2QuantumPressure(density, dx)
        pressureGradients = getCenteredGradient(P,   dx)

        #compute spatial gradients
        densityGradient   = getCenteredGradient(density, dx)
        for i in range(self.dimension):
            velocityGradients[i] = getCenteredGradient(velocities[i], dx)

        print("pressure gradients: ", pressureGradients[0], pressureGradients[1])

        #extrapolate
        for i in range(self.dimension):
            ddensity += velocities[i] * densityGradient[i] + density * velocityGradients[i][i]
            for j in range(self.dimension):
                dvelocities[i] += velocities[j] * velocityGradients[i][j]

            dvelocities[i] += pressureGradients[i]

        # extrapolate half-step in time
        density_prime    = density    - 0.5 * dt * ddensity
        velocities_prime = velocities - 0.5 * dt * dvelocities

        print("density prime, vx_prime, vy_prime", density_prime, velocities_prime[0], velocities_prime[1])

        # extrapolate in space to face centers
        density_ext    = extrapolateInSpaceToFace(density_prime, densityGradient, dx)

        for i in range(self.dimension):
            velocities_ext[i] = extrapolateInSpaceToFace(velocities_prime[i],  velocityGradients[i],  dx)

        print("extrapolated density, vxext, vyext ", density_ext[0][0], density_ext[0][1], density_ext[1][0], density_ext[1][1], 
        velocities_ext[0][0][0], velocities_ext[0][0][1], velocities_ext[0][1][0], velocities_ext[0][1][1], 
        velocities_ext[1][0][0], velocities_ext[1][0][1], velocities_ext[1][1][0], velocities_ext[1][1][1])


        # compute fluxes (Lax-Friedrichs)
        for i in range(self.dimension):
            if i == 0:
                massFluxes[i], momentaFluxes[i] = getFlux(density_ext, velocities_ext, dx, i, self.vmax)
            elif i == 1:
                massFluxes[i], momentaFluxes[i] = getFlux(density_ext, np.flip(velocities_ext, axis=0), dx, i, self.vmax)

        #momentaFluxes[0] contains momx_X, momy_X
        #momentaFluxes[0] contains momx_y, momy_y
        


        print("flux mass x, flux mass y", massFluxes[0], massFluxes[1])

        # update solution
        momenta   = velocities * density * volume
        density   = mapplyFluxes(density, massFluxes/volume, dx, dt)

        print("flux momx x, flux momx y, flux momy x, flux momy y ", momentaFluxes[0,0], momentaFluxes[1, 0], momentaFluxes[0,1], momentaFluxes[1, 1])
        #for i in range(self.dimension):
        if self.dimension == 1:
            momenta[0] = applyFluxes(momenta[0], momentaFluxes[0,0], momentaFluxes[0, 1], dx, dt)
        elif self.dimension == 2:
            momenta[0] = applyFluxes(momenta[0], momentaFluxes[0,0], momentaFluxes[1, 1], dx, dt)
            momenta[1] = applyFluxes(momenta[1], momentaFluxes[0,1], momentaFluxes[1, 0], dx, dt)

        print("Mass, Momx, Momy", density * volume, momenta)

        velocities = momenta / density

        ####KICK###

        ##Calculate gravitational potential
        self.potential          = self.computePotential(density)
        self.potentialGradients = getCenteredGradient(self.potential, dx)

        ## Add Source (half-step)

        velocities -= dt/2 * potentialGradients[i]

    def getDensity(self):
        return self.density

    def getPhase(self):
        return self.phase 

In [47]:
np.arange(16).reshape((2, 2, 2, 2))

array([[[[ 0,  1],
         [ 2,  3]],

        [[ 4,  5],
         [ 6,  7]]],


       [[[ 8,  9],
         [10, 11]],

        [[12, 13],
         [14, 15]]]])

In [52]:
import tests
import config 

def resetConfig(c):
    c["dt"] = 1e-3
    c["domainSize"] = 1
    c["resolution"] = 4
    c["timeOrder"] = 2
    c["stencilOrder"] = 4
    c["dimension"] = 2
    c["plotPhaseMod2"] = True
    #c["phaseYlim"] = [-3.14, 3.14]
    c["densityYlim"] = [0, 1]
    c["subregions"] = [[0.3, 0.3, 0.1, 0.1], [0.3, 0.3, 0.4, 0.4]]
    c["debug"] = False
    c["slowDown"] = 1
    c["tEnd"] = 2
    c["gravity"] = 1
    c["useAdaptiveSubregions"] = True
    c["maxSpeedC"] = 64/10
    c["useSlopeLimiting"] = True 
    c["fps"] = 10
    c["outputTimestep"] = True
    c["useAdaptiveTimestep"] = True 
    c["plotPhaseMod2"] = False
    c["phaseYlim"] = [-50, 50]
    c["nThreads"] = 4
    c["useHybrid"] = True

def test(xx, yy, dx, t):
    return tests.cosmological2D(xx, yy, dx, t, Lx=1, Ly=1, N=1, eps=0.5)


c = config.generateConfig(dt=1e-4, t0=0)

resetConfig(c)

In [53]:
ms = MUSCLHancock(c, test)
ms.step(1e-4)

Constructing scheme
Setting up grid
Reading in initial conditions
Setting up fourier grid
Vx, Vy:  [[ 0.19038389  0.05867217 -0.19038389 -0.05867217]
 [-0.28961299  0.08940278  0.28961299 -0.08940278]
 [-0.19038389 -0.05867217  0.19038389  0.05867217]
 [ 0.28961299 -0.08940278 -0.28961299  0.08940278]] [[-0.08940278 -0.28961299  0.08940278  0.28961299]
 [ 0.05867217 -0.19038389 -0.05867217  0.19038389]
 [ 0.08940278  0.28961299 -0.08940278 -0.28961299]
 [-0.05867217  0.19038389  0.05867217 -0.19038389]]
drift density, drift vx, drift vy  [[0.57719332 1.12475133 1.48703938 0.84388421]
 [0.70306012 0.88928943 1.30116874 1.07361347]
 [1.48703938 0.84388421 0.57719332 1.12475133]
 [1.30116874 1.07361347 0.70306012 0.88928943]] [[-9.51919431e-06 -2.93360832e-06  9.51919431e-06  2.93360832e-06]
 [ 1.44806497e-05 -4.47013909e-06 -1.44806497e-05  4.47013909e-06]
 [ 9.51919431e-06  2.93360832e-06 -9.51919431e-06 -2.93360832e-06]
 [-1.44806497e-05  4.47013909e-06  1.44806497e-05 -4.47013909e-06]

In [50]:
fs = FluidScheme(c, test)
fs.step(1e-4)

Constructing scheme
Setting up grid
Reading in initial conditions
Setting up fourier grid
0 0.0001
Vx, Vy:  [[ 0.19038389  0.05867217 -0.19038389 -0.05867217]
 [-0.28961299  0.08940278  0.28961299 -0.08940278]
 [-0.19038389 -0.05867217  0.19038389  0.05867217]
 [ 0.28961299 -0.08940278 -0.28961299  0.08940278]] [[-0.08940278 -0.28961299  0.08940278  0.28961299]
 [ 0.05867217 -0.19038389 -0.05867217  0.19038389]
 [ 0.08940278  0.28961299 -0.08940278 -0.28961299]
 [-0.05867217  0.19038389  0.05867217 -0.19038389]]
drift density, drift vx, drift vy  [[0.57719332 1.12475133 1.48703938 0.84388421]
 [0.70306012 0.88928943 1.30116874 1.07361347]
 [1.48703938 0.84388421 0.57719332 1.12475133]
 [1.30116874 1.07361347 0.70306012 0.88928943]] [[-9.51919431e-06 -2.93360832e-06  9.51919431e-06  2.93360832e-06]
 [ 1.44806497e-05 -4.47013909e-06 -1.44806497e-05  4.47013909e-06]
 [ 9.51919431e-06  2.93360832e-06 -9.51919431e-06 -2.93360832e-06]
 [-1.44806497e-05  4.47013909e-06  1.44806497e-05 -4.4701

Constructing scheme
Setting up grid
Reading in initial conditions
Setting up fourier grid
Vx, Vy:  [[ 0.19038389  0.05867217 -0.19038389 -0.05867217]
 [-0.28961299  0.08940278  0.28961299 -0.08940278]
 [-0.19038389 -0.05867217  0.19038389  0.05867217]
 [ 0.28961299 -0.08940278 -0.28961299  0.08940278]] [[-0.08940278 -0.28961299  0.08940278  0.28961299]
 [ 0.05867217 -0.19038389 -0.05867217  0.19038389]
 [ 0.08940278  0.28961299 -0.08940278 -0.28961299]
 [-0.05867217  0.19038389  0.05867217 -0.19038389]]
drift density, drift vx, drift vy  [[0.57719332 1.12475133 1.48703938 0.84388421]
 [0.70306012 0.88928943 1.30116874 1.07361347]
 [1.48703938 0.84388421 0.57719332 1.12475133]
 [1.30116874 1.07361347 0.70306012 0.88928943]] [[-9.51919431e-06 -2.93360832e-06  9.51919431e-06  2.93360832e-06]
 [ 1.44806497e-05 -4.47013909e-06 -1.44806497e-05  4.47013909e-06]
 [ 9.51919431e-06  2.93360832e-06 -9.51919431e-06 -2.93360832e-06]
 [-1.44806497e-05  4.47013909e-06  1.44806497e-05 -4.47013909e-06]

IndexError: index 2 is out of bounds for axis 0 with size 2

NameError: name 'f' is not defined

In [36]:
fs.step(1e-4)

0.0001 0.0001
Vx, Vy:  [[ 0.1903837   0.05867217 -0.1903837  -0.05867217]
 [-0.28961255  0.08940274  0.28961255 -0.08940274]
 [-0.1903837  -0.05867217  0.1903837   0.05867217]
 [ 0.28961255 -0.08940274 -0.28961255  0.08940274]] [[-0.08940274 -0.28961255  0.08940274  0.28961255]
 [ 0.05867217 -0.1903837  -0.05867217  0.1903837 ]
 [ 0.08940274  0.28961255 -0.08940274 -0.28961255]
 [-0.05867217  0.1903837   0.05867217 -0.1903837 ]]
drift density, drift vx, drift vy  [[0.57719393 1.12475133 1.48703859 0.84388435]
 [0.70306041 0.88928946 1.30116843 1.07361351]
 [1.48703859 0.84388435 0.57719393 1.12475133]
 [1.30116843 1.07361351 0.70306041 0.88928946]] [[ 0.00522457  0.00026248 -0.00113602 -0.00120882]
 [-0.00651478  0.00173949  0.00252115 -0.0007903 ]
 [-0.00113602 -0.00120882  0.00522457  0.00026248]
 [ 0.00252115 -0.0007903  -0.00651478  0.00173949]] [[-0.00201781 -0.00381919  0.00068718  0.00526133]
 [ 0.00102446 -0.00333972 -0.00048807  0.00268558]
 [ 0.00068718  0.00526133 -0.0020178

In [4]:
import numpy as np 

a = np.ones((3, 5, 5)) * 2
b = np.ones((5, 5)) * 3

b * a


	
	Momx -= dt * Mass * Vx
	Momy -= dt * Mass * Vy

	rho = Mass / vol
	vx  = Momx / rho / vol
	vy  = Momy / rho / vol


	rho = Mass / vol
	vx  = Momx / rho / vol
	vy  = Momy / rho / vol

array([[[6., 6., 6., 6., 6.],
        [6., 6., 6., 6., 6.],
        [6., 6., 6., 6., 6.],
        [6., 6., 6., 6., 6.],
        [6., 6., 6., 6., 6.]],

       [[6., 6., 6., 6., 6.],
        [6., 6., 6., 6., 6.],
        [6., 6., 6., 6., 6.],
        [6., 6., 6., 6., 6.],
        [6., 6., 6., 6., 6.]],

       [[6., 6., 6., 6., 6.],
        [6., 6., 6., 6., 6.],
        [6., 6., 6., 6., 6.],
        [6., 6., 6., 6., 6.],
        [6., 6., 6., 6., 6.]]])

In [ ]:
c = config.generateConfig(dt=1e-4, t0=0)

def resetConfig(c):
    c["dt"] = 1e-3
    c["domainSize"] = 12
    c["resolution"] = 16
    c["timeOrder"] = 2
    c["stencilOrder"] = 4
    c["dimension"] = 2
    c["plotPhaseMod2"] = True
    #c["phaseYlim"] = [-3.14, 3.14]
    c["densityYlim"] = [0, 1]
    c["subregions"] = [[0.3, 0.3, 0.1, 0.1], [0.3, 0.3, 0.4, 0.4]]
    c["debug"] = False
    c["slowDown"] = 1
    c["tEnd"] = 2
    c["gravity"] = 1
    c["useAdaptiveSubregions"] = True
    c["maxSpeedC"] = 64/10
    c["useSlopeLimiting"] = True 
    c["fps"] = 10
    c["outputTimestep"] = True
    c["useAdaptiveTimestep"] = True 
    c["plotPhaseMod2"] = False
    c["phaseYlim"] = [-50, 50]
    c["nThreads"] = 4
    c["useHybrid"] = True

resetConfig(c)

